***GENERATED CODE FOR manualclusteringmodel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerState', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '5095', 'mean': '', 'stddev': '', 'min': 'NY', 'max': 'WA', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerState'}, {'feature_label': 'CustomerState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StoreState', 'transformation_label': 'String Indexer'}], 'feature': 'StoreState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5095', 'mean': '', 'stddev': '', 'min': 'KS', 'max': 'WA', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StoreState'}, {'feature_label': 'StoreState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StoreState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationType', 'transformation_label': 'String Indexer'}], 'feature': 'LocationType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5095', 'mean': '', 'stddev': '', 'min': 'DC', 'max': 'Store', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationType'}, {'feature_label': 'LocationType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationType')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'deliveryDate', 'transformation_label': 'Extract Date'}], 'feature': 'deliveryDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'deliveryDate'}, {'feature_label': 'deliveryDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('deliveryDate')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent, k):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()
    model = KMeans(n_clusters=k).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        clusters = stageAttributes['totalClusters']

        bkmeans_model = kmeans(
            df=spark_DF, TrainingPercent=trainingPercent, k=clusters)


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run manualclusteringmodelHooks.ipynb
try:
	#sourcePreExecutionHook()

	cluteringmodel = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1632215175033_RoutingData_200921.csv', 'filename': '1632215175033_RoutingData_200921.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Biglynx_200921/RoutingData_200921.csv', 'viewFileName': 'RoutingData_200921.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run manualclusteringmodelHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(cluteringmodel,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "OrderLineItemId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "5095", "mean": "-6668400.13", "stddev": "1017927846.09", "min": "-2.146130102E9", "max": "2.146469551E9", "missing": "0"}, "updatedLabel": "OrderLineItemId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5095", "mean": "31.65", "stddev": "2.99", "min": "26.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5095", "mean": "-95.29", "stddev": "7.12", "min": "-122.0", "max": "-73.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerLon"}, {"transformationsData": [{"feature_label": "CustomerState", "transformation_label": "String Indexer"}], "feature": "CustomerState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5095", "mean": "", "stddev": "", "min": "NY", "max": "WA", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerState"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5095", "mean": "71597.58", "stddev": "17945.2", "min": "7202", "max": "98107", "missing": "0"}, "updatedLabel": "CustomerZip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5095", "mean": "32.95", "stddev": "3.51", "min": "29.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StoreLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5095", "mean": "-99.86", "stddev": "7.5", "min": "-123.0", "max": "-82.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StoreLon"}, {"transformationsData": [{"feature_label": "StoreState", "transformation_label": "String Indexer"}], "feature": "StoreState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5095", "mean": "", "stddev": "", "min": "KS", "max": "WA", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StoreState"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5095", "mean": "76638.63", "stddev": "9352.75", "min": "43232", "max": "98037", "missing": "0"}, "updatedLabel": "StoreZip"}, {"transformationsData": [{"feature_label": "LocationType", "transformation_label": "String Indexer"}], "feature": "LocationType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5095", "mean": "", "stddev": "", "min": "DC", "max": "Store", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationType"}, {"transformationsData": [{"feature_label": "deliveryDate", "transformation_label": "Extract Date"}], "feature": "deliveryDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "deliveryDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreCustomerDistance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5095", "mean": "655.81", "stddev": "740.85", "min": "0.0", "max": "3250.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StoreCustomerDistance"}, {"feature": "CustomerState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "5095", "mean": "0.21", "stddev": "0.76", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "CustomerState_stringindex..."}, {"feature": "StoreState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "5095", "mean": "0.4", "stddev": "0.96", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "StoreState_stringindexer_..."}, {"feature": "LocationType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "5095", "mean": "0.18", "stddev": "0.4", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "LocationType_stringindexe..."}, {"feature": "deliveryDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "5095", "mean": "14.51", "stddev": "8.44", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "deliveryDate_dayofmonth"}, {"feature": "deliveryDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "5095", "mean": "7.93", "stddev": "3.55", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "deliveryDate_month"}, {"feature": "deliveryDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "5095", "mean": "2018.86", "stddev": "0.35", "min": "2018", "max": "2019", "missing": "0"}, "updatedLabel": "deliveryDate_year"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run manualclusteringmodelHooks.ipynb
try:
	#mlPreExecutionHook()

	kmean = Clustering.run(autofe,spark,json.dumps( {"autoClustering": 0, "originalfile": "/FileStore/platform/sampleData/614c1a4c29a1e918b430cb54/part-00000-ebd20d3f-dd12-430a-8012-c70f918dab5e-c000.csv", "model": "K-Means", "TrainingPercent": 100, "totalClusters": 75}))

	#mlPostExecutionHook(kmean)

except Exception as ex: 
	logging.error(ex)
